In [1]:
!pip install -q crewai 'crewai[tools]' langchain_groq --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

In [3]:
import os
from google.colab import userdata

serper_api_key = userdata.get('serper_api')
os.environ["SERPER_API_KEY"] = serper_api_key

gemini_api_key = userdata.get('gemini_api')
os.environ["GEMINI_API_KEY"] = gemini_api_key

In [4]:
from crewai import LLM
gem_llm = LLM(model="gemini/gemini-1.5-flash", api_key=gemini_api_key)

In [5]:
from crewai_tools import (
    SerperDevTool,
    ScrapeWebsiteTool,
    FileReadTool
)

# Search tool for finding relevant Amazon help articles and policies
search_tool = SerperDevTool()

# Scraping tool to extract information from Amazon's help pages
scrape_tool = ScrapeWebsiteTool()

# Tool to read from a local knowledge base of common issues/solutions
knowledge_base = FileReadTool(file_path='knowledge_base.txt')

# You can initialize the tools with specific configurations if needed
# For example:
# search_tool = SerperDevTool(
#     params={
#         "num": 10,  # Number of search results
#         "site": "amazon.com/help"  # Restrict searches to Amazon help pages
#     }
# )

In [6]:
# Let's Set Up Our Agents

In [7]:
# Agent 1: Query Analyzer
query_analyzer = Agent(
    role="Customer Query Analyzer",
    goal="Accurately understand and classify customer queries, identify key issues, "
         "and determine the priority level of each case",
    tools=[search_tool],
    verbose=True,
    backstory=(
        "You are an expert in customer communication and query analysis, with "
        "deep knowledge of Amazon's product categories, services, and common "
        "customer issues. Your skill in understanding customer intent and "
        "identifying underlying problems is unmatched, helping to ensure each "
        "query is properly categorized and routed for resolution."
    ),
    llm=gem_llm
)

In [8]:
# Agent 2: Solution Researcher
solution_researcher = Agent(
    role="Solution Research Specialist",
    goal="Research and compile comprehensive solutions for customer queries using "
         "Amazon's knowledge base, policies, and previous case histories",
    tools=[search_tool, scrape_tool, knowledge_base],
    verbose=True,
    backstory=(
        "As a Solution Research Specialist, you excel at finding accurate and "
        "relevant solutions to customer problems. You have extensive knowledge of "
        "Amazon's policies, procedures, and best practices. Your expertise helps "
        "in discovering the most effective solutions, even for complex or unusual cases."
    ),
    llm=gem_llm
)

In [9]:
# Agent 3: Response Coordinator
response_coordinator = Agent(
    role="Customer Response Coordinator",
    goal="Craft personalized, clear, and effective responses to customer queries "
         "while ensuring compliance with Amazon's communication standards",
    tools=[search_tool, knowledge_base],
    verbose=True,
    backstory=(
        "You are an expert in customer communication, skilled at crafting clear, "
        "empathetic, and solution-focused responses. Your deep understanding of "
        "Amazon's voice and tone guidelines ensures that each response is not only "
        "helpful but also maintains Amazon's high standards of customer service."
    ),
    llm=gem_llm
)

In [10]:
# Agent 4: Escalation Specialist
escalation_specialist = Agent(
    role="Customer Issue Escalation Specialist",
    goal="Handle complex, high-priority cases and ensure proper escalation of "
         "issues that require specialized attention or immediate resolution",
    tools=[search_tool, scrape_tool, knowledge_base],
    verbose=True,
    backstory=(
        "You are a senior customer service specialist with expertise in handling "
        "critical and complex cases. Your deep understanding of Amazon's escalation "
        "protocols and problem-solving abilities enable you to manage sensitive "
        "situations, coordinate with relevant departments, and ensure swift resolution "
        "of urgent customer issues. You excel at identifying cases that need immediate "
        "attention and managing them through appropriate channels."
    ),
    llm=gem_llm
)

In [11]:
# Task for Query Analyzer Agent
analysis_task = Task(
    description=(
        "Analyze the customer query provided ({customer_query}) to: \n"
        "1. Identify the main issue and any secondary concerns\n"
        "2. Classify the query type (order issue, product question, etc.)\n"
        "3. Determine priority level based on urgency and impact\n"
        "4. Flag if the issue requires escalation\n"
        "Use the search tools to match with similar past cases and "
        "categorize appropriately."
    ),
    expected_output=(
        "A structured analysis including query classification, priority level, "
        "identified issues, and escalation recommendation if needed."
    ),
    agent=query_analyzer,
    async_execution=True
)

In [12]:
# Task for Solution Researcher Agent
research_task = Task(
    description=(
        "Research comprehensive solutions for the analyzed query ({customer_query}) by:\n"
        "1. Searching Amazon's knowledge base for relevant policies\n"
        "2. Finding similar resolved cases and their solutions\n"
        "3. Gathering relevant product information if needed\n"
        "4. Compiling step-by-step resolution steps\n"
        "Use all available tools to ensure thorough research."
    ),
    expected_output=(
        "A detailed solution package including relevant policies, "
        "step-by-step resolution steps, and any additional helpful resources."
    ),
    agent=solution_researcher,
    async_execution=True
)

In [13]:
# Task for Response Coordinator Agent
response_task = Task(
    description=(
        "Create a customer-friendly response using the research findings and "
        "solution package. The response should:\n"
        "1. Address the customer's concerns empathetically\n"
        "2. Provide clear, actionable solutions\n"
        "3. Follow Amazon's communication guidelines\n"
        "4. Include relevant links or resources\n"
        "Ensure the tone matches Amazon's voice and style."
    ),
    expected_output=(
        "A complete, personalized response that addresses all customer concerns "
        "in a clear, empathetic, and solution-focused manner."
    ),
    agent=response_coordinator,
    #async_execution=True
)

In [14]:
# Task for Escalation Specialist Agent
escalation_task = Task(
    description=(
        "Handle escalated cases that require special attention by:\n"
        "1. Reviewing the case details and previous interactions\n"
        "2. Determining the appropriate escalation path\n"
        "3. Coordinating with relevant internal departments\n"
        "4. Creating an action plan for resolution\n"
        "5. Monitoring the case progress\n"
        "Use all tools to ensure proper handling of sensitive cases."
    ),
    expected_output=(
        "A comprehensive escalation report including action plan, "
        "coordination details, and recommended next steps for resolution."
    ),
    agent=escalation_specialist,
    #async_execution=True
)

In [16]:
# Create Crew instance
customer_support_crew = Crew(
    agents=[query_analyzer,
            solution_researcher,
            response_coordinator,
            escalation_specialist],

    tasks=[analysis_task,
           research_task,
           response_task,
           escalation_task],

    verbose=True
)

In [18]:
# Define the input structure for customer support query
customer_support_inputs = {
    'customer_query': """I ordered a laptop (Order #ABC123456) three days ago with next-day delivery,
    but it hasn't arrived yet. The tracking hasn't updated in 48 hours.
    This is urgent as I need it for work.""",

    'order_details': {
        'order_number': 'ABC123456',
        'order_date': '2024-12-19',
        'expected_delivery': '2024-12-20',
        'product_category': 'Electronics',
        'shipping_method': 'Next-Day Delivery'
    },

    'customer_info': {
        'priority_status': 'Prime',
        'previous_cases': 'None'
    }
}

In [19]:
# Kickoff the crew with the inputs
result = customer_support_crew.kickoff(inputs=customer_support_inputs)

# Print or process the results
print("\nFinal Response:")
print(result)

# Agent: Solution Research Specialist
## Task: Research comprehensive solutions for the analyzed query (I ordered a laptop (Order #ABC123456) three days ago with next-day delivery,
    but it hasn't arrived yet. The tracking hasn't updated in 48 hours.
    This is urgent as I need it for work.) by:
1. Searching Amazon's knowledge base for relevant policies
2. Finding similar resolved cases and their solutions
3. Gathering relevant product information if needed
4. Compiling step-by-step resolution steps
Use all available tools to ensure thorough research.
# Agent: Customer Query Analyzer
## Task: Analyze the customer query provided (I ordered a laptop (Order #ABC123456) three days ago with next-day delivery,
    but it hasn't arrived yet. The tracking hasn't updated in 48 hours.
    This is urgent as I need it for work.) to: 
1. Identify the main issue and any secondary concerns
2. Classify the query type (order issue, product question, etc.)
3. Determine priority level based on urgency